# Vehicle Recommender Interactive Agent 


### We are going to Build a Car Recommender Agent

🎯 Core Features Implemented

**Multi-Agent Architecture**

- 4 Specialist Agents: Budget, Family, Luxury, Eco-friendly
- 1 Manager Agent: Orchestrates and routes to specialists
- 8 Function Tools: Direct inventory search capabilities



**Synthetic Data System**
- 6 Sample Vehicles: Diverse range (sedan, SUV, truck, electric, luxury)
- 4 Customer Personas: Different use cases and priorities
- Realistic Attributes: Price, MPG, safety, features, availability

### Imports

In [82]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
import asyncio
from agents import Agent, Runner, trace, function_tool, OpenAIChatCompletionsModel
from typing import Dict, List, Optional
import json
import pandas as pd
from datetime import datetime
import os
import nest_asyncio
import asyncio
from openai import AsyncOpenAI
from pydantic import BaseModel



In [83]:
nest_asyncio.apply()  # This MUST be called first!

 ### Jupyter  Notebook Compatibility  fix  for asyncio

####  Load environment variables

In [84]:
load_dotenv(override=True)
google_api_key = os.getenv('GOOGLE_API_KEY')
GEMINI_BASE_URL = "https://generativelanguage.googleapis.com/v1beta/openai/"
gemini_client = AsyncOpenAI(base_url=GEMINI_BASE_URL, api_key=google_api_key)
gemini_model = OpenAIChatCompletionsModel(model="gemini-2.0-flash", openai_client=gemini_client)

## Step 1: Synthetic Data Generation 

- Generate Synthetic Inventory 
- Generate Synthetic Persona 

In [85]:
def generate_synthetic_inventory():
    """Generate synthetic vehicle inventory data"""
    inventory_data = [
        {
            "id": "V001",
            "make": "Toyota",
            "model": "Camry",
            "year": 2024,
            "type": "Sedan",
            "price": 28000,
            "mpg_city": 28,
            "mpg_highway": 39,
            "seating_capacity": 5,
            "safety_rating": 5,
            "drivetrain": "FWD",
            "fuel_type": "Gasoline",
            "features": ["Backup Camera", "Bluetooth", "Lane Assist", "Adaptive Cruise Control"],
            "colors_available": ["White", "Black", "Silver", "Red"],
            "availability": "in_stock",
            "stock_count": 12,
            "category": "family",
            "description": "Reliable family sedan with excellent fuel economy and safety features"
        },
        {
            "id": "V002", 
            "make": "Honda",
            "model": "CR-V",
            "year": 2024,
            "type": "SUV",
            "price": 32000,
            "mpg_city": 27,
            "mpg_highway": 32,
            "seating_capacity": 5,
            "safety_rating": 5,
            "drivetrain": "AWD",
            "fuel_type": "Gasoline",
            "features": ["All-Wheel Drive", "Cargo Space", "Apple CarPlay", "Blind Spot Monitor"],
            "colors_available": ["Blue", "White", "Black", "Gray"],
            "availability": "in_stock",
            "stock_count": 8,
            "category": "suv",
            "description": "Versatile SUV perfect for families and outdoor adventures"
        },
        {
            "id": "V003",
            "make": "Tesla",
            "model": "Model 3",
            "year": 2024,
            "type": "Electric Sedan",
            "price": 42000,
            "mpg_city": 0,  # Electric
            "mpg_highway": 0,  # Electric
            "range_miles": 310,
            "seating_capacity": 5,
            "safety_rating": 5,
            "drivetrain": "RWD",
            "fuel_type": "Electric",
            "features": ["Autopilot", "Supercharger Network", "Over-the-air Updates", "Premium Audio"],
            "colors_available": ["White", "Black", "Blue", "Red"],
            "availability": "in_stock",
            "stock_count": 5,
            "category": "electric",
            "description": "Cutting-edge electric sedan with advanced technology and sustainability"
        },
        {
            "id": "V004",
            "make": "Ford",
            "model": "F-150",
            "year": 2024,
            "type": "Pickup Truck",
            "price": 38000,
            "mpg_city": 20,
            "mpg_highway": 26,
            "seating_capacity": 6,
            "safety_rating": 4,
            "drivetrain": "4WD",
            "fuel_type": "Gasoline",
            "features": ["Towing Capacity 13200lbs", "Bed Liner", "4WD", "Trailer Assist"],
            "colors_available": ["White", "Black", "Blue", "Red", "Silver"],
            "availability": "in_stock",
            "stock_count": 15,
            "category": "truck",
            "description": "America's best-selling truck with unmatched capability and durability"
        },
        {
            "id": "V005",
            "make": "BMW",
            "model": "X3",
            "year": 2024,
            "type": "Luxury SUV",
            "price": 48000,
            "mpg_city": 23,
            "mpg_highway": 29,
            "seating_capacity": 5,
            "safety_rating": 5,
            "drivetrain": "AWD",
            "fuel_type": "Gasoline",
            "features": ["Leather Seats", "Panoramic Sunroof", "Premium Audio", "Wireless Charging"],
            "colors_available": ["White", "Black", "Gray", "Blue"],
            "availability": "in_stock",
            "stock_count": 6,
            "category": "luxury",
            "description": "Luxury SUV combining performance, comfort, and prestige"
        },
        {
            "id": "V006",
            "make": "Hyundai",
            "model": "Elantra",
            "year": 2024,
            "type": "Compact Sedan",
            "price": 22000,
            "mpg_city": 33,
            "mpg_highway": 43,
            "seating_capacity": 5,
            "safety_rating": 4,
            "drivetrain": "FWD",
            "fuel_type": "Gasoline",
            "features": ["10-year Warranty", "Touchscreen", "Rear Camera", "Smart Key"],
            "colors_available": ["White", "Black", "Silver", "Blue"],
            "availability": "in_stock",
            "stock_count": 20,
            "category": "budget",
            "description": "Affordable compact sedan with excellent warranty and fuel efficiency"
        }
    ]
    
    # Save to JSON file
    with open('data/synthetic_inventory.json', 'w') as f:
        json.dump(inventory_data, f, indent=2)
    
    return inventory_data

In [86]:
def generate_customer_personas():
    """Generate synthetic customer personas for testing"""
    personas = [
        {
            "name": "Young Professional",
            "budget_max": 30000,
            "family_size": 1,
            "priorities": ["fuel_economy", "technology", "style"],
            "usage": "daily_commute",
            "preferences": ["compact", "sedan"],
            "test_query": "I need a fuel-efficient car for my daily commute under $30k"
        },
        {
            "name": "Growing Family",
            "budget_max": 40000,
            "family_size": 4,
            "priorities": ["safety", "space", "reliability"],
            "usage": "family_trips",
            "preferences": ["suv", "minivan"],
            "test_query": "I have two kids and need a safe, spacious vehicle for family trips"
        },
        {
            "name": "Eco-Conscious",
            "budget_max": 50000,
            "family_size": 2,
            "priorities": ["environmental", "technology", "efficiency"],
            "usage": "mixed",
            "preferences": ["electric", "hybrid"],
            "test_query": "I want an environmentally friendly car with the latest technology"
        },
        {
            "name": "Work Contractor",
            "budget_max": 45000,
            "family_size": 1,
            "priorities": ["capability", "durability", "utility"],
            "usage": "work_hauling",
            "preferences": ["truck", "van"],
            "test_query": "I need a truck for my construction business that can haul materials"
        }
    ]
    
    with open('data/customer_personas.json', 'w') as f:
        json.dump(personas, f, indent=2)
    
    return personas

# ===================

## Load Inventory data 

In [87]:
# Load inventory data
inventory_df = None

def load_inventory():
    """Load inventory data into pandas DataFrame"""
    global inventory_df
    try:
        with open('data/synthetic_inventory.json', 'r') as f:
            inventory_data = json.load(f)
        inventory_df = pd.DataFrame(inventory_data)
        return True
    except FileNotFoundError:
        # Generate synthetic data if file doesn't exist
        inventory_data = generate_synthetic_inventory()
        inventory_df = pd.DataFrame(inventory_data)
        return True
    except Exception as e:
        print(f"Error loading inventory: {e}")
        return False

## Tool Definitions 
- Search vehicles by budget 
- Search vehicles by type 
- Search vehicle by features 
- Get vehicle by fuel efficient features 
- Get vehicle by safety ratings 
- Get vehicle by vehicle id 
- Check inventory availability


In [88]:
@function_tool
def search_vehicles_by_budget(max_budget: int, min_budget: int = 0) -> List[Dict]:
    """Search vehicles within specified budget range"""
    if inventory_df is None:
        load_inventory()
    
    filtered = inventory_df[
        (inventory_df['price'] >= min_budget) & 
        (inventory_df['price'] <= max_budget) &
        (inventory_df['availability'] == 'in_stock')
    ]
    
    return filtered.to_dict('records')


In [89]:
@function_tool
def search_vehicles_by_type(vehicle_types: List[str]) -> List[Dict]:
    """Search vehicles by type (sedan, suv, truck, etc.)"""
    if inventory_df is None:
        load_inventory()
    
    # Convert to lowercase for matching
    types_lower = [t.lower() for t in vehicle_types]
    
    filtered = inventory_df[
        (inventory_df['type'].str.lower().isin(types_lower) | 
         inventory_df['category'].str.lower().isin(types_lower)) &
        (inventory_df['availability'] == 'in_stock')
    ]
    
    return filtered.to_dict('records')

In [90]:
@function_tool
def search_vehicles_by_features(required_features: List[str]) -> List[Dict]:
    """Search vehicles that have specific features"""
    if inventory_df is None:
        load_inventory()
    
    def has_features(vehicle_features, required):
        vehicle_features_lower = [f.lower() for f in vehicle_features]
        required_lower = [f.lower() for f in required]
        return any(req in ' '.join(vehicle_features_lower) for req in required_lower)
    
    filtered = inventory_df[
        inventory_df['features'].apply(lambda x: has_features(x, required_features)) &
        (inventory_df['availability'] == 'in_stock')
    ]
    
    return filtered.to_dict('records')

In [91]:
@function_tool
def get_fuel_efficient_vehicles(min_mpg: int = 30) -> List[Dict]:
    """Get vehicles with fuel efficiency above specified MPG"""
    if inventory_df is None:
        load_inventory()
    
    # For electric vehicles, consider them highly efficient
    filtered = inventory_df[
        (((inventory_df['mpg_city'] + inventory_df['mpg_highway']) / 2 >= min_mpg) |
         (inventory_df['fuel_type'] == 'Electric')) &
        (inventory_df['availability'] == 'in_stock')
    ]
    
    return filtered.to_dict('records')

In [92]:
@function_tool
def get_vehicles_by_safety_rating(min_rating: int = 4) -> List[Dict]:
    """Get vehicles with minimum safety rating"""
    if inventory_df is None:
        load_inventory()
    
    filtered = inventory_df[
        (inventory_df['safety_rating'] >= min_rating) &
        (inventory_df['availability'] == 'in_stock')
    ]
    
    return filtered.to_dict('records')

In [93]:
@function_tool
def get_vehicle_details(vehicle_id: str) -> Dict:
    """Get detailed information about a specific vehicle"""
    if inventory_df is None:
        load_inventory()
    
    vehicle = inventory_df[inventory_df['id'] == vehicle_id]
    if not vehicle.empty:
        return vehicle.iloc[0].to_dict()
    return {"error": "Vehicle not found"}

In [94]:
@function_tool
def compare_vehicles(vehicle_ids: List[str]) -> Dict:
    """Compare multiple vehicles side by side"""
    if inventory_df is None:
        load_inventory()
    
    vehicles = inventory_df[inventory_df['id'].isin(vehicle_ids)]
    comparison = {
        "vehicles": vehicles.to_dict('records'),
        "comparison_summary": {
            "price_range": f"${vehicles['price'].min():,} - ${vehicles['price'].max():,}",
            "best_mpg": vehicles.loc[vehicles['mpg_city'].idxmax(), 'model'] if not vehicles.empty else "N/A",
            "highest_safety": vehicles.loc[vehicles['safety_rating'].idxmax(), 'model'] if not vehicles.empty else "N/A"
        }
    }
    return comparison

In [95]:
@function_tool
def check_inventory_availability(vehicle_id: str) -> Dict:
    """Check real-time availability and stock count for a vehicle"""
    if inventory_df is None:
        load_inventory()
    
    vehicle = inventory_df[inventory_df['id'] == vehicle_id]
    if not vehicle.empty:
        return {
            "vehicle_id": vehicle_id,
            "availability": vehicle.iloc[0]['availability'],
            "stock_count": vehicle.iloc[0]['stock_count'],
            "estimated_delivery": "2-3 weeks" if vehicle.iloc[0]['stock_count'] > 0 else "6-8 weeks"
        }
    return {"error": "Vehicle not found"}

## Create Agents 


####  Prepare Tools List 

In [96]:
# Tools list for agents
vehicle_tools = [
    search_vehicles_by_budget,
    search_vehicles_by_type, 
    search_vehicles_by_features,
    get_fuel_efficient_vehicles,
    get_vehicles_by_safety_rating,
    get_vehicle_details,
    compare_vehicles,
    check_inventory_availability
]

### Create Budget Vehicles Focussed Agent 

In [97]:
# Budget-focused agent
budget_specialist = Agent(
    name="Budget Vehicle Specialist",
    instructions="""You are a budget-conscious vehicle specialist who helps customers find the best value vehicles within their price range. 
    
    You excel at:
    - Finding vehicles that maximize value for money
    - Identifying cost-effective options with good reliability
    - Explaining total cost of ownership
    - Highlighting vehicles with good resale value
    
    Always use the search tools to find current inventory and provide specific vehicle recommendations with pricing details.""",
    tools=vehicle_tools,
    model=gemini_model
)

### Family Focused Agent 

In [98]:

# Family-focused agent  
family_specialist = Agent(
    name="Family Vehicle Specialist", 
    instructions="""You are a family vehicle specialist who prioritizes safety, space, and practicality for families.
    
    You excel at:
    - Recommending vehicles based on family size and needs
    - Prioritizing safety ratings and family-friendly features
    - Considering cargo space, seating capacity, and convenience features
    - Understanding child safety requirements and accessibility
    
    Always use search tools to find vehicles that meet family requirements and emphasize safety ratings and practical features.""",
    tools=vehicle_tools,
    model=gemini_model
)

### Luxury Vehicle Agent 

In [99]:
# Performance/Luxury specialist
luxury_specialist = Agent(
    name="Luxury & Performance Specialist",
    instructions="""You are a luxury and performance vehicle specialist who focuses on premium features, advanced technology, and driving experience.
    
    You excel at:
    - Recommending vehicles with premium features and materials
    - Understanding performance specifications and driving dynamics  
    - Highlighting advanced technology and luxury amenities
    - Explaining the value proposition of premium vehicles
    
    Use search tools to find vehicles with luxury features and provide detailed explanations of premium benefits.""",
    tools=vehicle_tools,
    model=gemini_model
)

### Eco friendly vehicle specialist Agent 

In [100]:
# Eco-friendly specialist
eco_specialist = Agent(
    name="Eco-Friendly Vehicle Specialist",
    instructions="""You are an eco-friendly vehicle specialist focused on environmental impact, fuel efficiency, and sustainable transportation.
    
    You excel at:
    - Recommending electric, hybrid, and high-efficiency vehicles
    - Explaining environmental benefits and cost savings
    - Understanding charging infrastructure and range considerations
    - Calculating long-term fuel/energy cost savings
    
    Always prioritize vehicles with the best environmental credentials and use tools to find the most efficient options.""",
    tools=vehicle_tools,
    model=gemini_model
)

### Convert specialist agents to tools 


In [101]:
# Convert specialist agents to tools
budget_tool = budget_specialist.as_tool(
    tool_name="budget_specialist",
    tool_description="Get budget-focused vehicle recommendations and value analysis"
)

family_tool = family_specialist.as_tool(
    tool_name="family_specialist", 
    tool_description="Get family-oriented vehicle recommendations focusing on safety and practicality"
)

luxury_tool = luxury_specialist.as_tool(
    tool_name="luxury_specialist",
    tool_description="Get luxury and performance vehicle recommendations"
)

eco_tool = eco_specialist.as_tool(
    tool_name="eco_specialist",
    tool_description="Get eco-friendly and fuel-efficient vehicle recommendations"
)

### Main Orchestrating Agent 

In [102]:
# Main orchestrating agent
vehicle_recommendation_agent = Agent(
    name="Vehicle Recommendation Manager",
    instructions="""You are the main vehicle recommendation manager who helps customers find their perfect vehicle.

    Your process:
    1. Understand the customer's needs, budget, and priorities
    2. Determine which specialist agent(s) would be most helpful
    3. Use appropriate specialist tools to get recommendations
    4. Use direct search tools when needed for specific queries
    5. Synthesize recommendations from multiple sources
    6. Provide clear, actionable advice with specific vehicle suggestions

    Customer Priority Categories:
    - Budget-conscious → Use budget_specialist
    - Family needs → Use family_specialist  
    - Luxury/performance → Use luxury_specialist
    - Environmental/efficiency → Use eco_specialist

    Always provide:
    - At least 2-3 specific vehicle recommendations
    - Clear reasoning for each recommendation
    - Price information and key features
    - Next steps for the customer

    Be conversational, helpful, and thorough in your recommendations.""",
    tools=vehicle_tools + [budget_tool, family_tool, luxury_tool, eco_tool],
    model=gemini_model
)


### Define Testing  code for  Recommender Agent 


Here we develop a test framework with some pre loaded queries to test the Vehicle Recommendation  Agent 

In [103]:
async def test_multiple_queries_async():
    """Test multiple queries asynchronously"""
    
    if not load_inventory():
        generate_synthetic_inventory()
        load_inventory()
    
    test_queries = [
        "I need a reliable family car under $30k",
        "Show me luxury SUVs with advanced technology",
        "What's the most fuel-efficient vehicle you have?",
        "I need a truck for my construction business"
    ]
    
    print("=== TESTING MULTIPLE QUERIES ===\\n")
    
    for i, query in enumerate(test_queries, 1):
        print(f"Test {i}: {query}")
        print("-" * 50)
        
        try:
            with trace(f"Test Query {i}"):
                result = await Runner.run(vehicle_recommendation_agent, query)
                print(f"Response: {result.final_output}\\n")
        except Exception as e:
            print(f"Error: {e}\\n")
        
        print("=" * 80 + "\\n")



In [104]:
async def test_vehicle_agent():
    """Test the vehicle recommendation agent with sample queries"""
    
    # Ensure data is loaded
    load_inventory()
    generate_customer_personas()
    
    test_queries = [
        "I need a reliable family car with good fuel economy under $30k",
        "Looking for a luxury SUV with latest technology features",
        "What's the most environmentally friendly vehicle you have?",
        "I need a truck for my construction business that can haul materials",
        "Show me your most affordable cars with good safety ratings"
    ]
    
    print("=== TESTING VEHICLE RECOMMENDATION AGENT ===\n")
    
    for i, query in enumerate(test_queries, 1):
        print(f"Test {i}: {query}")
        print("-" * 50)
        
        with trace(f"Vehicle Recommendation Test {i}"):
            result = await Runner.run(vehicle_recommendation_agent, query)
            print(result.final_output)
        
        print("\n" + "="*80 + "\n")

### Define Test code framework for individual specialist Agents 


In [105]:
async def demo_specialist_agents():
    """Demo individual specialist agents"""
    
    load_inventory()
    
    print("=== TESTING SPECIALIST AGENTS ===\n")
    
    # Test budget specialist
    print("1. BUDGET SPECIALIST TEST")
    print("-" * 30)
    budget_query = "Find me the best value car under $25,000"
    
    with trace("Budget Specialist Demo"):
        result = await Runner.run(budget_specialist, budget_query)
        print(f"Query: {budget_query}")
        print(f"Response: {result.final_output}")
    
    print("\n" + "="*50 + "\n")
    
    # Test family specialist  
    print("2. FAMILY SPECIALIST TEST")
    print("-" * 30)
    family_query = "I have 3 kids and need a safe vehicle for school runs"
    
    with trace("Family Specialist Demo"):
        result = await Runner.run(family_specialist, family_query)
        print(f"Query: {family_query}")
        print(f"Response: {result.final_output}")
    
    print("\n" + "="*50 + "\n")

### Define code for running interactive demo of the Vehicle Reccomendation System 


In [106]:
async def run_interactive_demo_async():
    """Working interactive demo for Jupyter"""
    
    print("=== INTERACTIVE VEHICLE RECOMMENDATION DEMO ===")
    print("Ask me about vehicles! (type 'quit' to exit)\n")
    
    if not load_inventory():
        generate_synthetic_inventory()
        load_inventory()
    
    print(f"Inventory loaded: {len(inventory_df)} vehicles available\n")
    
    while True:
        user_input = input("You: ").strip()
        
        if user_input.lower() in ['quit', 'exit', 'stop']:
            print("Thanks for using the Vehicle Recommendation System!")
            break
            
        if not user_input:
            continue
            
        print("\nAgent: Thinking...")
        
        try:
            result = await Runner.run(vehicle_recommendation_agent, user_input)
            print(f"Agent: {result.final_output}\n")
            
        except Exception as e:
            print(f"Error: {e}\n")

### Main Driver Code 

In [107]:
# Create Directry if does not exist  
os.makedirs('data', exist_ok=True)


### Display Menu options 

In [47]:
print("Vehicle Recommendation Agent System")
print("=" * 40)
print("1. Testing individual components")
print("2. Running full system tests") 
print("3. Interactive demo")
print()

Vehicle Recommendation Agent System
1. Testing individual components
2. Running full system tests
3. Interactive demo



### Quick Test 

In [108]:
# Quick test - just run this cell
async def quick_test():
    load_inventory()
    result = await Runner.run(vehicle_recommendation_agent, "I need a family SUV under $35k")
    print(result.final_output)

await quick_test()

Okay, based on the initial search, a Honda CR-V (V002) looks like a good option.

**Honda CR-V (V002)**

*   **Price:** $32,000
*   **Key Features:** 5-star safety rating, All-Wheel Drive, ample cargo space, Apple CarPlay, Blind Spot Monitor

I still recommend answering the questions I asked earlier so I can provide a more tailored recommendation!



### Test Multiple Queries 

In [109]:
await test_multiple_queries_async()

=== TESTING MULTIPLE QUERIES ===\n
Test 1: I need a reliable family car under $30k
--------------------------------------------------
Response: Okay, I've got a couple of recommendations for you, keeping in mind your need for a reliable family car under $30k:

**1. Toyota Camry (around $28,000):**

*   **Reasoning:** The Toyota Camry is known for its reliability, excellent safety ratings, and good fuel economy. It comfortably seats 5 and comes standard with features like a backup camera, lane assist, and adaptive cruise control, which are great for family safety and convenience.
*   **Key Features:** 5-star safety rating, good MPG (28 city/39 highway), and essential safety features.

**2. Hyundai Elantra (around $22,000):**

*   **Reasoning:** The Hyundai Elantra is a more budget-friendly option that still offers a lot of value. It's fuel-efficient, comes with a fantastic 10-year warranty, and has features like a touchscreen and rear camera.
*   **Key Features:** Excellent warranty, go

### Test Specialist Agents 

In [110]:
async def test_specialist_agents_async():
    """Test individual specialist agents"""
    
    if not load_inventory():
        generate_synthetic_inventory()
        load_inventory()
    
    print("=== TESTING SPECIALIST AGENTS ===\\n")
    
    agents_and_queries = [
        (budget_specialist, "Find me the best value car under $25,000"),
        (family_specialist, "I have 3 kids and need a safe vehicle"),
        (luxury_specialist, "Show me your most premium vehicles"),
        (eco_specialist, "What's your most environmentally friendly option?")
    ]
    
    for agent, query in agents_and_queries:
        print(f"Testing {agent.name}")
        print(f"Query: {query}")
        print("-" * 40)
        
        try:
            result = await Runner.run(agent, query)
            print(f"Response: {result.final_output[:300]}...")
        except Exception as e:
            print(f"Error: {e}")
        
        print("\\n")

In [111]:
await test_specialist_agents_async()

=== TESTING SPECIALIST AGENTS ===\n
Testing Budget Vehicle Specialist
Query: Find me the best value car under $25,000
----------------------------------------
Response: OK. I found one vehicle that fits your criteria: The 2024 Hyundai Elantra, a compact sedan priced at $22,000. It gets 33 city / 43 highway MPG, seats 5, has a 4-star safety rating, and comes with a 10-year warranty. It also has a touchscreen, rear camera, and smart key.
...
\n
Testing Family Vehicle Specialist
Query: I have 3 kids and need a safe vehicle
----------------------------------------
Response: Okay, to recommend the best and safest vehicle for your family, I need a little more information.

1.  What is your budget for the vehicle?
2.  What type of vehicle are you interested in? (e.g., SUV, minivan, sedan, truck)
3.  Are there any specific features that are important to you? (e.g., third-r...
\n
Testing Luxury & Performance Specialist
Query: Show me your most premium vehicles
----------------------------------

### Complete System  Test 

In [112]:

async def complete_system_test():
    """Test tools through the agent instead of directly"""
    
    print("Testing Tools Through Agent")
    print("-" * 30)
    
    # Load inventory
    load_inventory()
    print(f"Inventory loaded: {len(inventory_df)} vehicles")
    
    # Test tools by asking the agent to use them
    test_queries = [
        "Show me cars between $20k and $30k",  # Tests budget search tool
        "What SUVs do you have?",              # Tests type search tool
        "Show me cars with high safety ratings" # Tests safety search tool
    ]
    
    for i, query in enumerate(test_queries, 1):
        print(f"\\n{i}. Testing: '{query}'")
        try:
            result = await Runner.run(vehicle_recommendation_agent, query)
            print(f"Success: {result.final_output[:100]}...")
        except Exception as e:
            print(f"Failed: {e}")

In [113]:
await complete_system_test()

Testing Tools Through Agent
------------------------------
Inventory loaded: 6 vehicles
\n1. Testing: 'Show me cars between $20k and $30k'
Success: Okay, I found two vehicles that fit your budget:

1.  **Toyota Camry (2024)**
    *   **Price:** $28...
\n2. Testing: 'What SUVs do you have?'
Success: OK, I found one SUV: the 2024 Honda CR-V. It's priced at $32,000, gets 27 MPG in the city and 32 MPG...
\n3. Testing: 'Show me cars with high safety ratings'
Success: What is the minimum safety rating you are looking for?
...


### test Interactive Demo 

In [ ]:
await run_interactive_demo_async()

=== INTERACTIVE VEHICLE RECOMMENDATION DEMO ===
Ask me about vehicles! (type 'quit' to exit)

Inventory loaded: 6 vehicles available



### End of Notebook